# **Notebook extracting line from proposal**


# Global import


In [ ]:
import json

import html

from pathlib import Path

from typing import List, Optional, Dict, Tuple, Union, Any

from collections import defaultdict

import tiktoken

from openai import OpenAI

from langchain.schema import BaseMessage, HumanMessage, SystemMessage

from app.cost.cost import CostTracker

from app.proposal_object.schemas import Devis

from app.proposal_object.path import ProposalObjectPaths

from app.proposal_object.proposal_object import ProposalObjectHandler

from config.logger_config import logger

from config.config import env_param


/Users/remillieux/Documents/Kleek/OCR/app/proposal_object/schemas.py:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if isinstance(row[0], str) and row[0].strip() == "Lot :":
/Users/remillieux/Documents/Kleek/OCR/app/proposal_object/schemas.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  isinstance(row[2], str)
/Users/remillieux/Documents/Kleek/OCR/app/proposal_object/schemas.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

# API Keys


In [2]:
AZURE_DI_ENDPOINT = "https://cleed-di.cognitiveservices.azure.com/"

AZURE_DI_KEY = "ae3ff9603ad64f81bf052eeee5a1ffae"


# Cost tracker


In [3]:
cost_tracker = CostTracker()


# Proposal


In [4]:
### Hard ###
FILE_NAME = "Devis DE2502-580 - LABOURDETTE - "


### Medium ###
# FILE_NAME = "Lot Electricité - JUILLARD - Devis_n°OF9206"

# FILE_NAME = "Devis - CHAINE CHAINE DES ARTISANS COTE SUD - DEVIS LARTIZIEN   1"

# FILE_NAME = "Devis_'OF9189'"

# FILE_NAME = "Devis DE2412-461 - GOURDON - "


### Easy ###
# FILE_NAME = "Devis - CHAINE CHAINE DES ARTISANS COTE SUD - Vouland   Seignosse"

proposal_path = ProposalObjectPaths(file_name=FILE_NAME)


# Utils


In [5]:
def token_counter(text: str) -> int:
    """Count the nb of token in a text"""

    encoding = tiktoken.get_encoding("cl100k_base")

    nb_token: int = len(encoding.encode(text))

    return nb_token


# **Document Intelligence**


# API


In [6]:
# from azure.core.credentials import AzureKeyCredential

# from azure.ai.documentintelligence import DocumentIntelligenceClient

# from azure.ai.documentintelligence.models import (
#     AnalyzeResult,
#     DocumentAnalysisFeature,
# )


# # ---------------------------------------------------------------------------
# # Analyse du document
# # ---------------------------------------------------------------------------

# document_intelligence_client = DocumentIntelligenceClient(
#     endpoint=AZURE_DI_ENDPOINT,
#     credential=AzureKeyCredential(AZURE_DI_KEY),
# )

# # 👉 Utilisation de toutes les features déjà demandées + nouvelles (langues, codes‑barres…)
# FEATURES = [
#     DocumentAnalysisFeature.STYLE_FONT,
#     DocumentAnalysisFeature.OCR_HIGH_RESOLUTION,
#     DocumentAnalysisFeature.KEY_VALUE_PAIRS,
# ]

# with open(proposal_path.proposal_path, "rb") as fh:
#     poller = document_intelligence_client.begin_analyze_document(
#         model_id="prebuilt-layout",
#         body=fh,
#         features=FEATURES,
#     )

# analyze_result: AnalyzeResult = poller.result()

# a_out = Path(proposal_path.proposal_ocr_json_path)

# a_out.parent.mkdir(parents=True, exist_ok=True)

# a_out.write_text(json.dumps(analyze_result.as_dict(), indent=2, ensure_ascii=False))

# print(f"✅ JSON écrit dans {a_out.resolve()}")

# a_out = Path(proposal_path.proposal_ocr_content_path)

# a_out.parent.mkdir(parents=True, exist_ok=True)

# a_out.write_text(analyze_result.content, encoding="utf-8")

# print(f"✅ TEXT écrit dans {a_out.resolve()}")


In [ ]:
import logging, statistics
from collections import defaultdict
from typing import Dict, Any, List

###############################################################################
# Réglages
###############################################################################
LOG_LEVEL = logging.INFO  # DEBUG pour traces complètes
Y_TOL = 0.06  # fusion verticale (∼6 % hauteur page)
X_TOL_COL = 6  # px : fusion colonnes voisines
TAB_SPACES = 3  # espaces insérés par « tabulation »
MAX_HLEVEL = 5  # ##### maxi en Markdown
logging.basicConfig(level=LOG_LEVEL, format="%(levelname)-8s %(message)s")


###############################################################################
# 1) Outils généraux
###############################################################################
def font_score(style: Dict[str, Any]) -> float:
    """
    Heuristique « taille + gras + italique » pour détecter les titres.
    Le style est celui stocké dans analyze_result["styles"][idx].
    """
    if not style:
        return 0.0
    s = style.get("fontSize", 10.0)
    # Azure : gras / ital => fontStyle == "bold/italic/bolditalic"
    fs = (style.get("fontStyle") or "").lower()
    if "bold" in fs:
        s += 2
    if "italic" in fs:
        s += 1
    return s


def pick_style_for_obj(
    obj: Dict[str, Any], styles: List[Dict[str, Any]]
) -> Dict[str, Any]:
    """
    Récupère le premier « span » de l’objet, et son index de style.
    """
    spans = obj.get("spans") or []
    if spans and "style" in spans[0]:
        idx = spans[0]["style"]
        if 0 <= idx < len(styles):
            return styles[idx]
    return {}  # style inconnu / absent


###############################################################################
# 2) Extraction de tous les blocs texte uniques
###############################################################################
def extract_blocks(ar: Dict[str, Any]) -> List[Dict[str, Any]]:
    """
    -> [{page, x, y, text, score}]
    """
    styles = ar.get("styles", [])
    sources = []
    # niveau « document »
    sources += ar.get("paragraphs", [])
    for t in ar.get("tables", []):
        sources += t.get("cells", t.get("tableCells", []))
    # sécurité : on scanne aussi pages/*
    for p in ar.get("pages", []):
        sources += p.get("paragraphs", [])
        for t in p.get("tables", []):
            sources += t.get("cells", t.get("tableCells", []))

    uniq, seen = [], set()
    for o in sources:
        if "content" not in o or not o.get("boundingRegions"):
            continue
        br = o["boundingRegions"][0]
        poly = br["polygon"]
        x = min(poly[i] for i in range(0, len(poly), 2))
        y = min(poly[i + 1] for i in range(0, len(poly), 2))
        pg = br.get("pageNumber", 1)
        txt = o["content"].strip()
        if not txt:
            continue
        key = (pg, round(x, 2), round(y, 2), txt)
        if key in seen:
            continue
        seen.add(key)
        st = pick_style_for_obj(o, styles)
        uniq.append(dict(page=pg, x=x, y=y, text=txt, score=font_score(st)))
    logging.info("🔎  %d blocs texte uniques", len(uniq))
    return uniq


###############################################################################
# 3) Regroupement en lignes & découverte des colonnes
###############################################################################
def group_lines(blocks: List[Dict]) -> Dict[int, List[List[Dict]]]:
    by_pg = defaultdict(list)
    for b in blocks:
        by_pg[b["page"]].append(b)

    lines_pg = {}
    for pg, blist in by_pg.items():
        blist.sort(key=lambda b: b["y"])
        lines = []
        for b in blist:
            for ln in lines:
                if abs(ln[0]["y"] - b["y"]) < Y_TOL:
                    ln.append(b)
                    break
            else:
                lines.append([b])
        for ln in lines:
            ln.sort(key=lambda b: b["x"])
        lines_pg[pg] = lines
    return lines_pg


def discover_columns(lines_pg: Dict[int, List[List[Dict]]]) -> List[float]:
    xs = sorted(b["x"] for pg in lines_pg.values() for ln in pg for b in ln)
    cols = []
    for x in xs:
        if not cols or abs(x - cols[-1]) > X_TOL_COL:
            cols.append(x)
    logging.info("📐  %d colonnes repérées", len(cols))
    return cols


def nearest_col(x: float, cols: List[float]) -> int:
    return min(range(len(cols)), key=lambda i: abs(x - cols[i]))


###############################################################################
# 4) Construction du Markdown hiérarchisé
###############################################################################
def to_markdown(ar: Dict[str, Any], out_path: str):
    blocks = extract_blocks(ar)
    if not blocks:
        logging.error("Aucun contenu exploitable.")
        return
    lines_pg = group_lines(blocks)
    cols = discover_columns(lines_pg)

    # N premiers scores distincts = niveaux de titre
    top_scores = sorted({b["score"] for b in blocks}, reverse=True)[:MAX_HLEVEL]
    score2lvl = {s: i + 1 for i, s in enumerate(top_scores)}

    with open(out_path, "w", encoding="utf-8") as fh:
        for pg in sorted(lines_pg):
            fh.write(f"\n<!-- ---------------- Page {pg} ---------------- -->\n\n")
            for ln in lines_pg[pg]:
                # Titre ?
                leader = min(ln, key=lambda b: b["x"])
                lvl = score2lvl.get(leader["score"])
                # Alignement horizontal :
                rendered, last_col = [], -1
                for b in ln:
                    col_idx = nearest_col(b["x"], cols)
                    rendered.append(
                        " " * ((col_idx - last_col - 1) * TAB_SPACES) + b["text"]
                    )
                    last_col = col_idx
                line_txt = "".join(rendered).rstrip()
                if lvl:
                    fh.write("#" * lvl + " " + line_txt + "\n\n")
                else:
                    fh.write(line_txt + "\n")
    logging.info("✅  Markdown écrit : %s", out_path)


###############################################################################
# Exemple d’utilisation
###############################################################################
# analyze_result est l’objet retourné par Azure :
ar = analyze_result.as_dict()
to_markdown(ar, "devis_reconstruit.md")


# **LLM formator**


# First structure analyze


In [12]:
from pydantic import BaseModel, Field


class NoeudStructureProduitsDevis(BaseModel):
    """
    Représente la structure des produits du devis.
    """

    nom: str = Field(
        ...,
        description="Nom du noeud de la structure des produits du devis",
    )

    sous_groupes_produits: Optional[List["NoeudStructureProduitsDevis"]] = Field(
        None,
        description="Liste des sous-groupes des produits du devis",
    )


class StructureProduitsDevis(BaseModel):
    groupes_produits: Optional[List["NoeudStructureProduitsDevis"]] = Field(
        None,
        description="Une ou plusieurs groupes de produits du devis. Si il n'y a qu'un seul groupe, la liste est vide.",
    )


PROPOSAL_SECTION_ANALYZER_SYSTEM_PROMPT = """Vous êtes le meilleur analyseur de devis du monde. Vous devez analyser un devis au format HTML pour déterminer la structure des catégories de produits de celui-ci, c'est-à-dire les différentes sections et sous-sections des produits du devis. Vous devez vérifier que le prix total de la catégorie correspond bien au prix des produits listés dans la catégorie. Respectez la structure originale du devis, ne tentez pas de fusionner des lignes. Ne comptez pas les nouvelles pages comme une nouvelle section. N'inventez aucune ligne."""

model = "o3"

client = OpenAI()

file = client.files.create(
    file=open(proposal_path.proposal_path, "rb"), purpose="user_data"
)

with open(
    proposal_path.proposal_html_path,
    "r",
    encoding="utf-8",
) as f:
    ocr_output = f.read()

completion = client.beta.chat.completions.parse(
    model=model,
    messages=[
        {
            "role": "system",
            "content": [
                {"type": "text", "text": PROPOSAL_SECTION_ANALYZER_SYSTEM_PROMPT}
            ],
        },
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "Voici le devis:\n\n" + ocr_output},
                {
                    "type": "file",
                    "file": {
                        "file_id": file.id,
                    },
                },
            ],
        },
    ],
    tools=[],
    store=False,
    response_format=StructureProduitsDevis,
    reasoning_effort="low",
)

proposal_structure: Optional[StructureProduitsDevis] = completion.choices[
    0
].message.parsed

if proposal_structure is None:
    logger.error("PROPOSAL => No structured proposal found.")


proposal_structure_json = proposal_structure.model_dump_json(indent=2)

print(f"Section analysis :\n{proposal_structure_json}")

print(f"Section analysis token :{token_counter(proposal_structure_json)}")

a_out = Path(proposal_path.proposal_section_analysis_path)

a_out.parent.mkdir(parents=True, exist_ok=True)

a_out.write_text(proposal_structure_json, encoding="utf-8")

print(f"✅ Section analysis written to {a_out.resolve()}")

cost_tracker.add_openai_query(
    model=model,
    nb_input_token=completion.usage.completion_tokens,
    nb_output_token=completion.usage.prompt_tokens,
    function_name="proposal_section_analyzer",
)

print(f"Total cost: {cost_tracker.cost.model_dump_json(indent=2)}")


Section analysis :
{
  "groupes_produits": [
    {
      "nom": "TRAVAUX SANITAIRES",
      "sous_groupes_produits": [
        {
          "nom": "Préparation",
          "sous_groupes_produits": null
        },
        {
          "nom": "Production d'eau chaude",
          "sous_groupes_produits": [
            {
              "nom": "CHAUFFE EAU ELECTRIQUE",
              "sous_groupes_produits": null
            }
          ]
        },
        {
          "nom": "Buanderie",
          "sous_groupes_produits": [
            {
              "nom": "LAVE-LINGE",
              "sous_groupes_produits": null
            }
          ]
        },
        {
          "nom": "WC 1",
          "sous_groupes_produits": [
            {
              "nom": "TOILETTES",
              "sous_groupes_produits": null
            },
            {
              "nom": "LAVE-MAINS",
              "sous_groupes_produits": null
            }
          ]
        },
        {
          "nom": "Cuisine",
 

# Agent


In [13]:
from app.agent.agent import Agent

PROPOSAL_ANALYZER_SYSTEM_PROMPT = """Vous êtes le meilleur analyseur de devis du monde. Vous devez reconstruire le devis à partir de l'OCR. Utilisez les polygones pour trouver la place des éléments. Pour chaque produit du devis, vous devez très précisément noter son label, sa description, le lot auquel il appartient son prix HT la quantité et la TVA. Le produits peuvent être grouper par catégorie et ou contenir des sous catégories de produits."""

model = "o3"  # "gpt-4o-mini" or "o4-mini" or "gpt-4o" or "gpt-4o-2024-08-06"


class ProposalAnalyzer:
    """Proposal analyzer"""

    def __init__(self, cost_tracker: CostTracker) -> None:
        self.agent: Agent = Agent(
            api_key=env_param.OPENAI_API_KEY,
            model_name=model,
            system_promt=PROPOSAL_ANALYZER_SYSTEM_PROMPT,
            nb_retry=2,
            temprature=1 if model in ["o4-mini", "o3"] else 0,
            timeout=200,
            name="proposal_analyzer",
            cost_tracker=cost_tracker,
            reasoning_effort="low",
        )

    async def analyze(self, chat_historic: List[BaseMessage]) -> Optional[Devis]:
        """Strucutre the proposal"""

        proposal_structured: Optional[Devis] = await self.agent.run(
            response_format=Devis, chat_historic=chat_historic
        )

        return proposal_structured

    async def structure_proposal(
        self, raw_proposal: str, section_analysis: str
    ) -> Optional[Devis]:
        """Check if the client is spam"""

        chat_historic = [
            SystemMessage(content=PROPOSAL_ANALYZER_SYSTEM_PROMPT),
            HumanMessage(
                "Voici un premier analyse de la sturcutre du devis :\n\n"
                + section_analysis
            ),
            HumanMessage(content=raw_proposal),
        ]

        proposal_structured: Optional[Devis] = await self.analyze(chat_historic)

        if proposal_structured is None:
            logger.info("PROPOSAL => No structured proposal found.")

            return None

        return proposal_structured


# Run agent


In [14]:
with open(
    proposal_path.proposal_html_path,
    "r",
    encoding="utf-8",
) as f:
    raw_proposal = f.read()

with open(
    proposal_path.proposal_section_analysis_path,
    "r",
    encoding="utf-8",
) as f:
    section_analysis_raw = f.read()

section_analysis: Optional[StructureProduitsDevis] = (
    StructureProduitsDevis.model_validate_json(section_analysis_raw)
)

quote_analyzer = ProposalAnalyzer(cost_tracker=cost_tracker)

proposal: Optional[Devis] = await quote_analyzer.structure_proposal(
    raw_proposal=raw_proposal,
    section_analysis=section_analysis.model_dump_json(indent=2),
)

if proposal is None:
    logger.error("PROPOSAL => No structured proposal found.")
    exit(1)

proposal_json = proposal.model_dump_json(indent=2)

print(f"PROPOSAL => {proposal_json}")

a_out = Path(proposal_path.proposal_object_predicted_path)

a_out.parent.mkdir(parents=True, exist_ok=True)

a_out.write_text(proposal_json)

print(f"✅ OBJECT prédit écrit dans {a_out.resolve()}")


print(f"Total cost: {cost_tracker.cost.model_dump_json(indent=2)}")


2025-06-20 19:51:13 INFO -> [app/performances/time_counter.py.time_counter.async_wrapper.37]:                                        PERFORMANCES => Function 'run' executed in : 36.960 seconds


PROPOSAL => {
  "devis_id": "DE2502-580",
  "devis_date": "13/03/2025",
  "devis_total_ht": 15036.99,
  "devis_total_ttc": 16540.69,
  "devis_total_tva": 1503.7,
  "devis_coup_additionnel": 6.97,
  "devis_produits": [
    {
      "label": "TRAVAUX SANITAIRES",
      "description": "",
      "quantite": 1.0,
      "unitee_quantite": null,
      "price_unitaire_ht": 6824.9,
      "tva": "TVA 10%",
      "cout_additionnel": null,
      "sous_produits": [
        {
          "label": "Préparation",
          "description": "",
          "quantite": 1.0,
          "unitee_quantite": null,
          "price_unitaire_ht": 1395.0,
          "tva": "TVA 10%",
          "cout_additionnel": null,
          "sous_produits": [
            {
              "label": "Mise en sécurité et pose d'un robinet de chantier",
              "description": "",
              "quantite": 1.0,
              "unitee_quantite": "U",
              "price_unitaire_ht": 135.0,
              "tva": "TVA 10%",
           

# Proposal object


In [15]:
ProposalObjectHandler(proposal_path=proposal_path).analyze_proposal_object()


2025-06-20 19:51:26 INFO -> [app/proposal_object/proposal_object.py.proposal_object.convert_to_text.49]:                             ✅ PROPOSAL TEXT write in /Users/remillieux/Documents/Kleek/OCR/data/proposal_text/Devis DE2502-580 - LABOURDETTE - .txt


2025-06-20 19:51:26 INFO -> [app/proposal_object/proposal_object.py.proposal_object.convert_to_pdf.65]:                              ✅ PROPOSAL PDF write in /Users/remillieux/Documents/Kleek/OCR/data/proposal_pdf_object/Devis DE2502-580 - LABOURDETTE - .pdf
2025-06-20 19:51:26 INFO -> [app/proposal_object/proposal_object.py.proposal_object.convert_to_proposal_lines.94]:                   ✅ PROPOSAL LINES write in /Users/remillieux/Documents/Kleek/OCR/data/proposal_object_line/Devis DE2502-580 - LABOURDETTE - .txt
2025-06-20 19:51:26 INFO -> [app/proposal_object/proposal_object.py.proposal_object.validate_proposal_object.116]:                   {
  "computed_total_ht": 9659.01,
  "computed_total_tva": 965.23,
  "computed_total_ttc": 10624.24,
  "computed_total_cout_additionnel": 0.0,
  "logs": [
    "📑 Démarrage de la vérification des prix pour le devis :",
    "🔽 Entrée dans « TRAVAUX SANITAIRES » (HT déclaré = 6824.90)",
    "    🔽 Entrée dans « TRAVAUX SANITAIRES > Préparation » (HT 

arraybcimunnm = [
    "|    PACK   |   TVA   |                                   LABEL                                    |                                    DESCRIPTION                                     | QTY | UNIT | UNIT_PRICE_HT | TOTAL_HT |",
    "|-----------|---------|----------------------------------------------------------------------------|------------------------------------------------------------------------------------|-----|------|---------------|----------|",
    "| SANITAIRE | TVA 10% |                             TRAVAUX SANITAIRES                             |                                                                                    |  1  |      |    6824.90    | 6824.90  |",
    "| SANITAIRE | TVA 10% |                                Préparation                                 |                                                                                    |  1  |      |    1395.00    | 1395.00  |",
    "| SANITAIRE | TVA 10% |             Mise en s